In [4]:
!pip install cleverhans

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent
import numpy as np

In [6]:
# Load a pretrained model (ResNet-18)
model = torchvision.models.resnet18(pretrained=True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
# Define a loss function and optimizer
loss_fn = nn.CrossEntropyLoss()

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet-18 expects 224x224 images
    transforms.ToTensor(),
])


In [8]:
# Use the test set for evaluation
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified


In [9]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [10]:
# PGD Attack parameters
eps = 0.3  # Perturbation amount
eps_iter = 0.01  # Step size per iteration
nb_iter = 40  # Number of iterations

# Evaluate the model on adversarial examples
correct = 0
total = 0

for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    # Generate adversarial examples using PGD
    adv_images = projected_gradient_descent(model, images, eps=eps, eps_iter=eps_iter, nb_iter=nb_iter, norm=np.inf)

    # Forward pass the adversarial images through the model
    outputs = model(adv_images)
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the model on adversarial images: {} %'.format(100 * correct / total))

Accuracy of the model on adversarial images: 1.07 %
